In [20]:
import pandas as pd
import numpy as np
import pyspark
from collections import Counter
import matplotlib.pyplot as plt

In [6]:
data = pd.read_csv('the-reddit-climate-change-dataset-comments.csv')
data = data.dropna() # drop any rows with missing values

In [19]:
# counter = Counter(data['subreddit.nsfw'])
# print(counter)

# print(data['score'])
print(data['body'].iloc[20340])

The issues you're describing are not a sunk cost fallacy. This is just the way capitalism and democracy are intended to work. 

If there's an issue where everyone really needs to be on the same page, capitalism and democracy just don't work well. In capitalism, as soon as clean energy is at a similar price to fossil fuels, there will be a transition. Businesses love certainty, and constantly fluctuating energy prices cause them many problems, even for oil companies. In democracy all the people who will be hurt by a transition away from fossil fuels are entitled to a voice. Rapid change, no matter how good the change is, always leaves people behind. 

These people have no reason to believe government will support them when they're left behind. What did the US do for auto workers when factories left, or when coal mines shut down? What about those making horse carriages in the early 1900s?  See the history of every industrial and technological revolution in the US. The wealthy and powerfu